# Langgraph

In [1]:
import os
import traceback
import concurrent.futures
import json
import csv

from dotenv import load_dotenv
from openai import OpenAI
from typing import TypedDict, Dict, Any, List
from langgraph.graph import StateGraph, END

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

LLM_MODEL_FOR_PARSING = "gpt-4o"        
LLM_MODEL_FOR_DECOMPOSITION = "gpt-4o" 

## 요구사항 정제/추출 이후 상세설명이 달린 데이터 필요

In [2]:
# 작업 1: 요구사항 분류
def generate_classification_prompt_text(description, detailed_description, module):
    return f"""
당신은 차세대 정보시스템 구축 프로젝트에서 요구사항을 분석하고, 아래 기준에 따라 대분류, 중분류, 소분류를 분류하는 전문가입니다.

다음 요구사항 설명을 읽고 각 분류 항목에 맞게 분류해 주세요:

[요구사항 설명]
{description}

[상세 설명]
{detailed_description}

[담당 모듈]
{module}

[분류 기준]
1. **대분류**: 차세대 정보시스템 업무 수준
   예시: 수신, 여신, 부대/대행, 통합고객 등

2. **중분류**: 단위업무 시스템 수준
   예시: 예금, 신탁, 상담신청, 심사승인 등

3. **소분류**: 단위업무 시스템 하위 수준 (업무 프로세스 3~4레벨)
   ※ 소분류는 3레벨 분류가 어려운 경우 선택적으로 작성해도 무방함

아래 형식으로 정확히 출력하세요 (불필요한 설명 없이):

대분류: <텍스트>
중분류: <텍스트>
소분류: <텍스트 또는 '해당 없음'>

※ 유의사항:
- 반드시 대분류 → 중분류 → 소분류 순으로 작성
- 각 분류명은 명확하고 직관적인 한국어 명사형 표현을 사용할 것
- 기존 분류 체계가 없으므로, 의미적으로 유사한 요구사항끼리 논리적으로 묶어서 계층화할 것
- 불필요한 설명 없이 위 형식만 출력
"""

def classify_requirement_logic(description: str, detailed_description: str, module: str) -> Dict[str, str]:
    prompt = generate_classification_prompt_text(description, detailed_description, module)
    try:
        response = client.chat.completions.create(
            model="gpt-4", # 또는 gpt-4-turbo, gpt-3.5-turbo 등 사용 가능한 모델
            messages=[
                {"role": "system", "content": "당신은 소프트웨어 분석 및 분류 전문가입니다."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3
        )
        content = response.choices[0].message.content or ""
        lines = [line.strip() for line in content.splitlines() if ":" in line]

        def extract_value(prefix):
            for line in lines:
                if line.startswith(prefix):
                    parts = line.split(":", 1)
                    if len(parts) == 2:
                        return parts[1].strip()
            return "미분류"

        return {
            "category_large": extract_value("대분류"),
            "category_medium": extract_value("중분류"),
            "category_small": extract_value("소분류"),
        }
    except Exception as e:
        print(f"Error in classify_requirement_logic: {e}")
        return {
            "category_large": "Error",
            "category_medium": "Error",
            "category_small": "Error"
        }


In [3]:
# 작업 2: 난이도 평가
def generate_difficulty_prompt_text(description, detailed_description, module):
    return f"""
당신은 소프트웨어 요구사항의 기술적 구현 난이도를 분석하고 평가하는 **수십 년 경력의 베테랑 개발 팀장 또는 시스템 아키텍트**입니다. 제시된 요구사항의 **기술적 복잡성, 필요한 리서치 및 학습량, 구현에 필요한 공수, 외부 시스템과의 연동 복잡성, 테스트의 난해함, 그리고 잠재적인 리스크** 등을 종합적으로 고려하여 난이도를 '상', '중', '하' 중 하나로 매우 신중하고 일관성 있게 평가해야 합니다.

다음은 시스템에 대한 요구사항입니다:

[요구사항 설명]
{description}

[상세 설명]
{detailed_description}

[담당 모듈]
{module}

요구사항을 분석한 뒤, 다음의 **판단 가이드라인과 세부 평가 기준**을 참고하여 난이도를 평가하세요:

**[판단 가이드라인: 난이도에 영향을 미치는 주요 요소]**
1.  **요구사항의 명확성 및 구체성:** 요구사항이 모호하거나 해석의 여지가 많을수록 분석 및 설계 단계부터 어려움이 추가되어 난이도가 상승합니다.
2.  **기술적 생소함 및 복잡도:** 새로운 프로그래밍 언어, 프레임워크, 라이브러리, 알고리즘의 도입이 필요하거나, 기존에 다뤄보지 않은 매우 복잡한 기술적 구현이 요구될 경우 난이도가 높습니다.
3.  **시스템 연동의 범위 및 복잡도:** 연동해야 할 내부/외부 시스템의 수가 많거나, 연동 방식(API, 프로토콜 등)이 복잡하거나, 연동 대상 시스템의 문서화가 미흡하거나 기술 지원이 원활하지 않을 경우 난이도가 크게 상승합니다.
4.  **데이터 처리 및 마이그레이션의 복잡성:** 처리해야 할 데이터의 양이 매우 방대하거나, 데이터 구조가 복잡하거나, 기존 시스템과의 데이터 정합성 유지 및 마이그레이션 작업이 까다로울 경우 난이도가 높습니다.
5.  **기존 시스템에 대한 영향 (Side Effect):** 요구사항 구현으로 인해 기존 시스템의 다른 부분에 예상치 못한 영향을 미칠 가능성이 높고, 이로 인해 광범위한 테스트와 수정이 필요할 경우 난이도가 증가합니다.
6.  **테스트의 복잡성 및 용이성:** 단위 테스트, 통합 테스트, 시스템 테스트 시나리오가 복잡하거나, 테스트 환경 구축이 어렵거나, 테스트 데이터 생성이 까다로운 경우 난이도가 높습니다.
7.  **비기능적 요구사항의 달성 난이도:** 매우 높은 수준의 성능(응답 시간, 처리량), 보안(암호화, 접근 제어), 안정성, 확장성 등의 비기능적 요구사항을 만족시켜야 한다면 기술적 도전 과제가 많아져 난이도가 상승합니다.
8.  **유지보수성 고려:** 향후 유지보수가 용이하도록 코드를 구조화하고 문서화하는 데 추가적인 노력이 많이 필요할 것으로 예상되면 난이도에 반영될 수 있습니다.

**[난이도 평가 기준]**

* **상 (H, High):**
    * **판단 근거:** 위의 판단 가이드라인 중 **다수 항목에서 높은 수준의 복잡성 또는 불확실성**이 확인되거나, **특정 한두 요소가 프로젝트 일정에 심각한 지연을 초래할 만큼 매우 치명적인 기술적 장벽**을 포함하는 경우.
    * **특징적 상황:**
        * 핵심 아키텍처 변경 또는 검증되지 않은 신기술의 광범위한 도입/연구가 필요함.
        * 매우 복잡한 알고리즘 설계 및 구현, 또는 전례 없는 수준의 시스템 연동이 요구됨.
        * 요구사항의 불확실성이 극도로 높아 초기 분석/설계 단계에서부터 상당한 시간과 리서치, PoC(Proof of Concept)가 필요함.
        * 구현 실패의 리스크가 높거나, 성공하더라도 사전에 계획된 개발 일정을 현저히 초과할 가능성이 매우 농후함.
        * 해결을 위해 팀 내 최고 수준의 전문가 투입 또는 외부 전문 컨설팅이 필요할 수 있음.
    * **일정 영향:** 자체 개발 일정에 **심각한 차질(예: 주요 마일스톤의 상당한 지연, 예정된 리소스 초과 등)**을 초래할 정도의 매우 높은 노력과 시간이 필요하며, 별도의 핵심 과제로 관리되어야 하는 수준.

* **중 (M, Medium):**
    * **판단 근거:** 판단 가이드라인 중 **일부 항목에서 중간 정도의 복잡성**이 관찰되거나, 해결해야 할 몇 가지 기술적 고려사항 및 도전 과제가 명확히 존재하는 경우.
    * **특징적 상황:**
        * 익숙한 기술 스택을 기반으로 하지만 새로운 기능을 개발하거나 기존 기능에 대한 상당한 수정이 필요함.
        * 일부 복잡한 비즈니스 로직, 혹은 예측 가능한 범위 내의 기술적 문제 해결이 요구됨.
        * 내부 모듈 간의 복잡한 상호작용 또는 비교적 잘 정의된 외부 시스템과의 연동 작업이 포함됨.
        * 어느 정도의 분석/설계 시간이 필요하며, 구현 중 예상치 못한 이슈가 발생할 수 있으나 관리 가능한 수준.
    * **일정 영향:** 집중적인 노력과 계획적인 접근을 통해 **자체 개발 일정 내에 충분히 완수 가능**하나, 일정 내에서도 타이트하거나 약간의 도전이 따를 수 있는 수준.

* **하 (L, Low):**
    * **판단 근거:** 판단 가이드라인의 대부분 항목에서 복잡성이 낮거나, 기술적으로 **매우 명확하고 직접적인 해결 방법**이 존재하는 경우.
    * **특징적 상황:**
        * 기존 기능의 단순 버그 수정, UI 텍스트 변경, 경미한 디자인 조정, 이미 잘 구축된 컴포넌트의 재활용 또는 매우 간단한 로직의 추가.
        * 새로운 기술 학습이나 복잡한 분석/설계 과정이 거의 불필요하며, 구현 경로가 명확함.
        * 타 시스템과의 연동이 없거나 매우 단순하며, 테스트가 용이함.
    * **일정 영향:** 현재 진행 중인 다른 작업과 병행하거나, **자체 개발 일정의 일부분으로 특별한 부담 없이 충분히 흡수**되어 별도의 추가 공수 산정 없이 진행 가능한 수준.

아래와 같이 **정확히 이 형식**으로만 출력하세요 (불필요한 설명 없이):

난이도: <상|중|하>
"""

def get_difficulty_logic(description: str, detailed_description: str, module: str) -> str:
    prompt = generate_difficulty_prompt_text(description, detailed_description, module)
    try:
        response = client.chat.completions.create(
            model="gpt-4", # 또는 gpt-4-turbo, gpt-3.5-turbo 등 사용 가능한 모델
            messages=[
                {"role": "system", "content": "당신은 소프트웨어 분석 전문가입니다."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.4
        )
        content = response.choices[0].message.content or ""
        difficulty = next((line.split(":")[1].strip() for line in content.splitlines() if "난이도" in line), "중") # 기본값을 '중'으로 설정
        return difficulty
    except Exception as e:
        print(f"Error in get_difficulty_logic: {e}")
        return "Error"


In [4]:
# 작업 3: 중요도 평가
def generate_importance_prompt_text(description, detailed_description, module):
    return f"""
당신은 소프트웨어 요구사항을 분석하여 중요도를 판단하는 **매우 숙련되고 비판적인 시스템 분석가**입니다. 제시된 기준과 판단 가이드라인에 따라 각 요구사항의 중요도를 '상', '중', '하' 중 하나로 **극도로 신중하고 일관성 있게** 평가해야 합니다. **'상' 등급은 매우 제한적으로 사용되어야 함**을 명심하십시오.

다음은 시스템에 대한 요구사항입니다:

[요구사항 설명]
{description}

[상세 설명]
{detailed_description}

[담당 모듈]
{module}

요구사항을 분석한 뒤, 아래의 **세분화된 기준과 판단 가이드라인**에 따라 중요도를 평가하세요:

**[판단 가이드라인]**
1.  **가장 먼저 '상' (Critical)에 해당하는지 극도로 보수적으로 판단합니다.** 이 요구사항이 없으면 시스템 자체가 완전히 무너지거나 법적/보안적으로 회복 불가능한 치명적 문제가 발생하는지 자문하십시오. **대부분의 요구사항은 '상'에 해당하지 않을 가능성이 높습니다.**
2.  '상'이 아니라면, '중' (Important)에 해당하는지 검토합니다.
3.  '상'도 '중'도 아니라면 '하' (Useful)로 평가합니다.
4.  요구사항의 단어나 문구에 현혹되지 말고, **실제 시스템 전체에 미치는 파급 효과와 해당 요구사항 실패 시의 구체적인 결과를 기준으로 냉정하게 판단**하십시오. 모든 요구사항이 중요해 보일 수 있지만, 자원은 한정되어 있으므로 상대적인 중요도를 엄격히 구분해야 합니다.

**[중요도 평가 기준]**

* **상 (C, Critical):**
    * **판단 기준:** 해당 요구사항의 미구현이 **시스템 전체의 핵심 기능 마비, 서비스 불가능 상태 초래, 심각한 법적/규제적 문제 야기, 대규모 중요 데이터의 영구적 손실 또는 오염, 회복 불가능한 치명적 보안 사고 발생**과 같이 프로젝트의 존립을 위협하거나 시스템 전체의 실패를 의미하는 경우에만 해당합니다. **대체 수단이 전혀 없거나, 그 영향이 조직/서비스 전체에 즉각적이고 치명적인 경우**에만 극히 제한적으로 부여합니다.
    * **'상'이 아닌 경우 (예시):** 단순히 "필수적"이라고 언급되거나, 중요한 기능처럼 보이더라도, 위와 같은 수준의 치명적이고 즉각적인 결과로 이어지지 않는다면 '상'으로 평가해서는 안 됩니다. 예를 들어, 특정 기능의 부재가 큰 불편을 야기하지만 시스템의 다른 핵심 기능은 정상 동작한다면 '상'이 아닙니다.

* **중 (I, Important):**
    * **판단 기준:** 시스템의 기능적 완성도, 운영 효율성, 사용자 만족도에 **상당한 영향을 미치지만, 그것이 없다고 해서 시스템 전체가 즉시 마비되거나 사용 불가능 상태가 되지는 않는 경우**입니다. 미구현 시 서비스 중단까지는 아니지만, 주요 사용자의 큰 불편을 초래하거나, 기업의 수익/평판에 측정 가능한 부정적 영향을 미치거나, 핵심 업무 프로세스에 심각한 차질을 주는 경우 해당됩니다.
    * **'중'이 아닌 경우 (예시):** 사소한 불편함, 일부 제한된 사용자에게만 영향, 또는 있으면 좋지만 없어도 큰 지장이 없는 경우는 '중'이 아닙니다.

* **하 (U, Useful):**
    * **판단 기준:** 구현되면 유용하고 사용자 경험을 개선할 수 있지만, 미구현되어도 시스템의 핵심 기능, 안정성, 보안 및 주요 사용자 그룹의 전반적인 만족도에 **심각한 영향을 주지 않는 사항**입니다. 약간의 불편함이 있거나, 특정 소수의 사용자에게만 영향을 미치거나, 다른 기능으로 비교적 쉽게 대체 가능하거나, 장기적으로 고려할 만한 개선 사항인 경우 해당됩니다.

아래와 같이 **정확히 이 형식**으로만 출력하세요 (불필요한 설명 없이):

중요도: <상|중|하>
"""

def get_importance_logic(description: str, detailed_description: str, module: str) -> str:
    prompt = generate_importance_prompt_text(description, detailed_description, module)
    try:
        response = client.chat.completions.create(
            model="gpt-4", # 또는 gpt-4-turbo, gpt-3.5-turbo 등 사용 가능한 모델
            messages=[
                {"role": "system", "content": "당신은 소프트웨어 분석 전문가입니다."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.4
        )
        content = response.choices[0].message.content or ""
        importance = next((line.split(":")[1].strip() for line in content.splitlines() if "중요도" in line), "중") # 기본값을 '중'으로 설정
        return importance
    except Exception as e:
        print(f"Error in get_importance_logic: {e}")
        return "Error"


In [5]:
# --- LangGraph 부분 시작 ---
class RequirementAnalysisState(TypedDict, total=False):
    # Fields from your input JSON
    id: str
    type: str
    description: str
    detailed_description: str
    acceptance_criteria: str
    responsible_module: str
    parent_id: str
    source_pages: List[int]

    # Fields populated by LLM tasks
    # classification: Dict[str, str]  # 이 줄을 삭제합니다.
    category_large: str             # 다음 세 줄을 추가합니다.
    category_medium: str
    category_small: str
    difficulty: str
    importance: str

    # Final aggregated output for each item
    combined_results: Dict[str, Any]
# LangGraph 노드 정의

# 병렬 실행을 위한 통합 노드
# 이 노드 내에서 ThreadPoolExecutor를 사용하여 세 가지 작업을 병렬로 실행합니다.
def node_parallel_assessments(state: RequirementAnalysisState) -> Dict[str, Any]:
    description = state["description"]
    detailed_description = state["detailed_description"]
    module = state["responsible_module"]

    # results 딕셔너리는 각 로직 함수의 반환 값을 임시 저장합니다.
    classification_dict: Dict[str, str] = {} # 타입 힌트 추가
    difficulty_str: str = ""
    importance_str: str = ""

    with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
        future_classify = executor.submit(classify_requirement_logic, description, detailed_description, module)
        future_difficulty = executor.submit(get_difficulty_logic, description, detailed_description, module)
        future_importance = executor.submit(get_importance_logic, description, detailed_description, module)

        classification_dict = future_classify.result()
        difficulty_str = future_difficulty.result()
        importance_str = future_importance.result()

    # 반환하는 딕셔너리가 RequirementAnalysisState의 개별 키와 일치하도록 수정
    return {
        "category_large": classification_dict.get("category_large", "미분류"),
        "category_medium": classification_dict.get("category_medium", "미분류"),
        "category_small": classification_dict.get("category_small", "미분류"),
        "difficulty": difficulty_str,
        "importance": importance_str,
    }

# 모든 결과를 취합하는 노드
def node_combine_results(state: RequirementAnalysisState) -> Dict[str, Any]:

    # combined_data는 현재 state의 모든 항목 (입력 + LLM 결과)을 포함해야 합니다.
    # 'combined_results' 키 자체는 최종 상태 업데이트를 위한 것이므로 제외합니다.
    
    combined_data_for_output: Dict[str, Any] = {}
    for key, value in state.items():
        if key != "combined_results": # 'combined_results'는 이 노드가 채울 필드이므로 복사 대상에서 제외
            combined_data_for_output[key] = value

    # 이 노드는 상태의 'combined_results' 필드를 업데이트할 딕셔너리를 반환합니다.
    return {"combined_results": combined_data_for_output}

In [6]:
def convert_json_to_csv(json_file_path, csv_file_path):
    """
    주어진 JSON 파일에서 데이터를 읽어 지정된 CSV 양식으로 변환하여 저장합니다.

    Args:
        json_file_path (str): 입력 JSON 파일의 경로입니다.
        csv_file_path (str): 출력 CSV 파일의 경로입니다.
    """
    # CSV 헤더 정의
    csv_headers = [
        "요구사항 ID",
        "요구사항유형(기능/비기능)",
        "대분류",
        "중분류",
        "소분류",
        "요구사항 명",
        "요구사항 설명",
        "중요도",
        "난이도"
    ]

    try:
        with open(json_file_path, 'r', encoding='utf-8') as json_file:
            try:
                requirements_data = json.load(json_file)
            except json.JSONDecodeError as e:
                print(f"오류: JSON 파일 '{json_file_path}' 파싱 중 오류 발생: {e}")
                return
    except FileNotFoundError:
        print(f"오류: JSON 파일 '{json_file_path}'를 찾을 수 없습니다.")
        return
    except Exception as e:
        print(f"오류: JSON 파일 '{json_file_path}'을 여는 중 오류 발생: {e}")
        return

    # JSON 데이터가 리스트 형태인지 확인 (일반적으로 여러 요구사항이 리스트 안에 있음)
    if not isinstance(requirements_data, list):
        # 단일 JSON 객체인 경우, 리스트로 감싸서 처리
        if isinstance(requirements_data, dict):
            requirements_data = [requirements_data]
            print(f"정보: 입력된 JSON 데이터가 단일 객체이므로 리스트로 변환하여 처리합니다.")
        else:
            print(f"오류: JSON 데이터의 최상위 구조가 리스트 또는 단일 객체가 아닙니다 (타입: {type(requirements_data)}).")
            return
            
    try:
        with open(csv_file_path, 'w', newline='', encoding='utf-8-sig') as csv_file: # utf-8-sig는 Excel에서 한글 깨짐 방지
            writer = csv.writer(csv_file)
            
            # 1. CSV 헤더 쓰기
            writer.writerow(csv_headers)

            # 2. JSON 데이터 순회하며 CSV 행 쓰기
            for req in requirements_data:
                if not isinstance(req, dict):
                    print(f"경고: 요구사항 데이터 리스트 내에 딕셔너리가 아닌 항목 발견. 건너<0xEB><0x9A><0x84>니다: {req}")
                    continue
                
                # 각 필드의 값을 가져오고, 없는 경우 빈 문자열로 대체 (get 메소드 활용)
                row_to_write = [
                    req.get("id", ""),
                    req.get("type", ""),
                    req.get("category_large", ""),
                    req.get("category_medium", ""),
                    req.get("category_small", ""),
                    req.get("description", ""),  # "요구사항 명"
                    req.get("detailed_description", ""),  # "요구사항 설명"
                    req.get("importance", ""),
                    req.get("difficulty", "")
                ]
                writer.writerow(row_to_write)
        
        print(f"성공: '{json_file_path}' 파일의 내용이 '{csv_file_path}' CSV 파일로 성공적으로 변환되었습니다.")

    except IOError as e:
        print(f"오류: CSV 파일 '{csv_file_path}'을(를) 쓰거나 여는 중 I/O 오류 발생: {e}")
    except Exception as e:
        print(f"CSV 변환 중 예기치 않은 오류 발생: {e}")

In [7]:
# 그래프 빌더 생성
workflow = StateGraph(RequirementAnalysisState)

# 노드 추가
workflow.add_node("parallel_processor", node_parallel_assessments)
workflow.add_node("final_combiner", node_combine_results) # 병렬 처리된 결과를 최종 정리

# 엣지 연결
workflow.set_entry_point("parallel_processor")
workflow.add_edge("parallel_processor", "final_combiner")
workflow.add_edge("final_combiner", END)


# 그래프 컴파일
app = workflow.compile()

In [8]:
# --- 새로운 메인 로직 ---
def load_requirements_from_json(filepath: str) -> List[Dict[str, str]]:
    """지정된 경로에서 JSON 파일을 로드하여 요구사항 목록을 반환합니다."""
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            data = json.load(f)
            if isinstance(data, list):
                return data
            else:
                print(f"Error: JSON 파일 ({filepath})이 리스트 형태가 아닙니다.")
                return []
    except FileNotFoundError:
        print(f"Error: JSON 파일을 찾을 수 없습니다 - {filepath}")
        return []
    except json.JSONDecodeError:
        print(f"Error: JSON 파일 디코딩 중 오류 발생 - {filepath}")
        return []
    except Exception as e:
        print(f"Error loading JSON file {filepath}: {e}")
        return []

def save_results_to_json(results: List[Dict[str, Any]], filepath: str):
    """결과 목록을 지정된 경로에 JSON 파일로 저장합니다."""
    try:
        with open(filepath, 'w', encoding='utf-8') as f:
            json.dump(results, f, indent=2, ensure_ascii=False)
        print(f"\n결과가 성공적으로 {filepath} 파일에 저장되었습니다.")
    except Exception as e:
        print(f"Error saving results to JSON file {filepath}: {e}")

def main():
    """
    JSON 파일에서 요구사항을 로드하고, 각 요구사항을 처리한 후,
    모든 결과를 단일 JSON 파일로 저장합니다.
    """

    input_json_path = "docs/test_lv3.json"
    output_json_path = "docs/test_lv4.json"

    requirements_to_process = load_requirements_from_json(input_json_path)

    if not requirements_to_process:
        print("처리할 요구사항이 없습니다. 스크립트를 종료합니다.")
        return

    all_combined_results = []
    total_requirements = len(requirements_to_process)

    print(f"\n총 {total_requirements}개의 요구사항 처리를 시작합니다...")

    for i, req_data in enumerate(requirements_to_process):
        print(f"\n[{i+1}/{total_requirements}] 처리 중: '{req_data.get('description', 'N/A')[:70]}...'")

        # LangGraph에 전달할 초기 상태 구성
        # req_data는 input JSON 파일의 각 객체입니다.
        inputs_for_graph: RequirementAnalysisState = {
            # Map all fields from req_data to the state
            "id": req_data.get("id"),
            "type": req_data.get("type"),
            "description": req_data.get("description", "내용 없음"), # Used by LLMs
            "detailed_description": req_data.get("detailed_description", "상세 내용 없음"), # Used by LLMs
            "acceptance_criteria": req_data.get("acceptance_criteria"),
            "responsible_module": req_data.get("responsible_module"), # Original module field
            "parent_id": req_data.get("parent_id"),
            "source_pages": req_data.get("source_pages"),
            
            # classification, difficulty, importance, combined_results는 그래프 내에서 채워집니다.
        }

        try:
            # LangGraph 실행
            final_state = app.invoke(inputs_for_graph)

            if final_state and "combined_results" in final_state:
                all_combined_results.append(final_state["combined_results"])
            else:
                print(f"    ⚠️ [{i+1}/{total_requirements}] 요구사항 처리 후 'combined_results'를 찾을 수 없습니다. Skipping.")
                # 에러 상황에 대한 대체 결과 추가 가능
                error_result = {
                    "input_description": inputs_for_graph["description"],
                    "error": "Processing failed or combined_results not found in final state.",
                    "details": final_state
                }
                all_combined_results.append(error_result)


        except Exception as e:
            print(f"    ❌ [{i+1}/{total_requirements}] 요구사항 처리 중 오류 발생: {e}")
            traceback.print_exc()
            # 오류 발생 시에도 입력 정보를 포함한 에러 메시지를 결과에 추가
            error_result = {
                "input_description": inputs_for_graph["description"],
                "input_detailed_description": inputs_for_graph["detailed_description"],
                "input_module": inputs_for_graph["module"],
                "error": str(e),
                "classification": "Error",
                "difficulty": "Error",
                "importance": "Error"
            }
            all_combined_results.append(error_result)

    # 모든 결과를 JSON 파일로 저장
    save_results_to_json(all_combined_results, output_json_path)

In [9]:
if __name__ == "__main__":
    print("스크립트 시작...")
    main()
    input_json_file = 'docs/test_lv4.json'
    output_csv_file = 'output/final_srs.csv'

    # --- 실행 ---
    # 입력 파일이 존재하는지 먼저 확인
    if os.path.exists(input_json_file):
        convert_json_to_csv(input_json_file, output_csv_file)
    else:
        raise FileNotFoundError(f"입력 JSON 파일이 존재하지 않습니다: '{input_json_file}'")

스크립트 시작...

총 6개의 요구사항 처리를 시작합니다...

[1/6] 처리 중: 'UI/UX 디자인 가이드라인 개발...'

[2/6] 처리 중: '컴포넌트 라이브러리 구축...'

[3/6] 처리 중: '사용자 인터페이스 표준화...'

[4/6] 처리 중: '접근성 표준 준수...'

[5/6] 처리 중: '반응형 디자인 구현...'

[6/6] 처리 중: '사용자 피드백 수집 및 반영...'

결과가 성공적으로 docs/test_lv4.json 파일에 저장되었습니다.
성공: 'docs/test_lv4.json' 파일의 내용이 'output/final_srs.csv' CSV 파일로 성공적으로 변환되었습니다.
